In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers


In [ ]:

# Пример данных InputLog
InputLog = [
    {'buttonKey': 1, 'timestamp': 1, 'isWorkMode': True},
    {'buttonKey': 2, 'timestamp': 2, 'isWorkMode': False},
    # добавьте остальные записи...
]


In [ ]:

# Преобразуем данные в DataFrame
df = pd.DataFrame(InputLog)


In [ ]:

# Убедимся, что timestamp отсортирован
df = df.sort_values(by='timestamp')


In [ ]:

# Создание временных окон (180 секунд)
window_size = 180  # seconds
X, y = [], []

for i in range(len(df)):
    end_time = df['timestamp'].iloc[i]
    start_time = end_time - window_size
    
    # Извлечение данных из окна
    window = df[(df['timestamp'] >= start_time) & (df['timestamp'] <= end_time)]
    
    if len(window) > 0:
        # Преобразование окна в структурированный массив
        X.append(window[['buttonKey']].values.flatten())
        # Определяем метку для каждого окна
        y.append(int(window['isWorkMode'].any()))  # True, если есть хотя бы одно True

X = np.array(X)
y = np.array(y)


In [ ]:

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:

# Создание модели
model = keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(10, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:

# Обучение модели
model.fit(X_train, y_train, epochs=10, batch_size=32)


In [ ]:

# Оценка модели
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy:.4f}')


In [ ]:

# Пример предсказания
sample_input = np.array([[1, 2, 3, 4, 5]])  # пример нового окна
predicted = model.predict(sample_input)
print(f'Predicted isWorkMode: {predicted[0][0] > 0.5}')  # True если предсказание > 0.5
